# Nodo XXI

# Noticias

In [7]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
import time

def create_driver(headless=False):
    options = Options()
    if headless:
        options.add_argument("--headless")
    driver = webdriver.Chrome(options=options)
    return driver

# Inicializar el WebDriver
driver = create_driver()

# URL base
driver.get("https://www.nodoxxi.cl/noticias/")
time.sleep(3)  # Esperar a que cargue la página

# Lista para almacenar los artículos
all_articles = []

# Encontrar el contenedor principal de los artículos
try:
    container = driver.find_element(By.CSS_SELECTOR, "div.et_pb_portfolio_items_wrapper.clearfix")
    articles = container.find_elements(By.CSS_SELECTOR, "div.et_pb_portfolio_item")
    
    for article in articles:
        # Extraer título y enlace
        title_tag = article.find_element(By.CSS_SELECTOR, "h2.entry-title a")
        title = title_tag.text.strip()
        link = title_tag.get_attribute("href")
        
        # Extraer fecha
        try:
            date_tag = article.find_element(By.CSS_SELECTOR, "span.ppp-blog-post-date")
            date = date_tag.text.strip().replace(" |", "")
        except:
            date = None
        
        # Extraer categorías (Producto)
        try:
            category_tags = article.find_elements(By.CSS_SELECTOR, "span.ppp-blog-post-categories a")
            categories = ", ".join([cat.text.strip() for cat in category_tags])
        except:
            categories = None
        
        # Guardar el artículo
        all_articles.append({
            "Título": title,
            "Enlace": link,
            "Fecha": date,
            "Producto": categories
        })

except Exception as e:
    print(f"Error al extraer artículos: {e}")

# Cerrar el WebDriver
driver.quit()

# Convertir a DataFrame
n21 = pd.DataFrame(all_articles)

# Mostrar resultados
n21

,Título,Enlace,Fecha,Producto
0,"A cuatro años de la revuelta social, volver a ...",https://www.nodoxxi.cl/noticias/2023/a-cuatro-...,18 Oct 2023,"Estado Social, Política, Sociedad"
1,"El país no necesita al CAE, el CAE necesita so...",https://www.nodoxxi.cl/noticias/opinion/2023/e...,7 Oct 2023,Educación
2,Conmemoración 50 años del Golpe de Estado: ¿Ha...,https://www.nodoxxi.cl/noticias/2023/conmemora...,23 Jul 2023,"Derechos Humanos, Política"
3,Presentación de la IPN Nº 10.887,https://www.nodoxxi.cl/noticias/2023/presentac...,23 Jul 2023,"Consejo Constitucional, Estado Social"
4,DECLARACIÓN INICIATIVAS POPULARES CIUDADANAS,https://www.nodoxxi.cl/noticias/2023/declaraci...,11 Jul 2023,Consejo Constitucional
...,...,...,...,...
125,,https://www.nodoxxi.cl/noticias/2018/se-realiz...,,
126,,https://www.nodoxxi.cl/noticias/opinion/2018/l...,,
127,,https://www.nodoxxi.cl/noticias/2018/nodo-xxi-...,,
128,,https://www.nodoxxi.cl/noticias/2017/luis-thie...,,


# Investigación

In [3]:
import pandas as pd
from bs4 import BeautifulSoup

html = """
<div class="tagcloud"><a href="https://www.nodoxxi.cl/temas/comision-experta/" class="tag-cloud-link tag-link-534 tag-link-position-1" style="font-size: 9.5180722891566pt;" aria-label="Comisión experta (2 elementos)">Comisión experta</a>
<a href="https://www.nodoxxi.cl/temas/consejo-constitucional/" class="tag-cloud-link tag-link-527 tag-link-position-2" style="font-size: 12.638554216867pt;" aria-label="Consejo Constitucional (6 elementos)">Consejo Constitucional</a>
<a href="https://www.nodoxxi.cl/temas/convencion/" class="tag-cloud-link tag-link-478 tag-link-position-3" style="font-size: 9.5180722891566pt;" aria-label="Convención (2 elementos)">Convención</a>
<a href="https://www.nodoxxi.cl/temas/convencion-constitucional/" class="tag-cloud-link tag-link-477 tag-link-position-4" style="font-size: 14.831325301205pt;" aria-label="Convención Constitucional (12 elementos)">Convención Constitucional</a>
<a href="https://www.nodoxxi.cl/temas/convencion-constituyente/" class="tag-cloud-link tag-link-476 tag-link-position-5" style="font-size: 13.481927710843pt;" aria-label="Convención Constituyente (8 elementos)">Convención Constituyente</a>
<a href="https://www.nodoxxi.cl/temas/cuidados/" class="tag-cloud-link tag-link-485 tag-link-position-6" style="font-size: 10.530120481928pt;" aria-label="Cuidados (3 elementos)">Cuidados</a>
<a href="https://www.nodoxxi.cl/temas/cultura/" class="tag-cloud-link tag-link-13 tag-link-position-7" style="font-size: 9.5180722891566pt;" aria-label="Cultura (2 elementos)">Cultura</a>
<a href="https://www.nodoxxi.cl/temas/derechos-humanos/" class="tag-cloud-link tag-link-15 tag-link-position-8" style="font-size: 13.903614457831pt;" aria-label="Derechos Humanos (9 elementos)">Derechos Humanos</a>
<a href="https://www.nodoxxi.cl/temas/economia/" class="tag-cloud-link tag-link-16 tag-link-position-9" style="font-size: 15.421686746988pt;" aria-label="Economía (14 elementos)">Economía</a>
<a href="https://www.nodoxxi.cl/temas/educacion/" class="tag-cloud-link tag-link-17 tag-link-position-10" style="font-size: 17.361445783133pt;" aria-label="Educación (25 elementos)">Educación</a>
<a href="https://www.nodoxxi.cl/temas/estado-social/" class="tag-cloud-link tag-link-541 tag-link-position-11" style="font-size: 9.5180722891566pt;" aria-label="Estado Social (2 elementos)">Estado Social</a>
<a href="https://www.nodoxxi.cl/temas/estado-social-consejo-constitucional/" class="tag-cloud-link tag-link-542 tag-link-position-12" style="font-size: 8pt;" aria-label="Estado Social (1 elemento)">Estado Social</a>
<a href="https://www.nodoxxi.cl/temas/feminismo/" class="tag-cloud-link tag-link-114 tag-link-position-13" style="font-size: 17.78313253012pt;" aria-label="Feminismo (28 elementos)">Feminismo</a>
<a href="https://www.nodoxxi.cl/temas/medio-ambiente/" class="tag-cloud-link tag-link-161 tag-link-position-14" style="font-size: 8pt;" aria-label="Medio Ambiente (1 elemento)">Medio Ambiente</a>
<a href="https://www.nodoxxi.cl/temas/nueva-constitucion/" class="tag-cloud-link tag-link-507 tag-link-position-15" style="font-size: 15.843373493976pt;" aria-label="Nueva Constitución (16 elementos)">Nueva Constitución</a>
<a href="https://www.nodoxxi.cl/temas/pensiones/" class="tag-cloud-link tag-link-160 tag-link-position-16" style="font-size: 11.373493975904pt;" aria-label="Pensiones (4 elementos)">Pensiones</a>
<a href="https://www.nodoxxi.cl/temas/plebiscito/" class="tag-cloud-link tag-link-499 tag-link-position-17" style="font-size: 10.530120481928pt;" aria-label="Plebiscito (3 elementos)">Plebiscito</a>
<a href="https://www.nodoxxi.cl/temas/plebiscito-de-salida/" class="tag-cloud-link tag-link-503 tag-link-position-18" style="font-size: 9.5180722891566pt;" aria-label="Plebiscito de salida (2 elementos)">Plebiscito de salida</a>
<a href="https://www.nodoxxi.cl/temas/politica/" class="tag-cloud-link tag-link-124 tag-link-position-19" style="font-size: 22pt;" aria-label="Política (91 elementos)">Política</a>
<a href="https://www.nodoxxi.cl/temas/sociedad/" class="tag-cloud-link tag-link-297 tag-link-position-20" style="font-size: 15.421686746988pt;" aria-label="Sociedad (14 elementos)">Sociedad</a>
<a href="https://www.nodoxxi.cl/temas/trabajo/" class="tag-cloud-link tag-link-159 tag-link-position-21" style="font-size: 15.168674698795pt;" aria-label="Trabajo (13 elementos)">Trabajo</a></div>
"""

soup = BeautifulSoup(html, 'html.parser')

data = []
for a_tag in soup.find_all('a'):
    href = a_tag['href']
    nombre = a_tag.text
    data.append({'Nombre': nombre, 'Enlace': href})

inv = pd.DataFrame(data)

inv

# Para guardar el DataFrame en un archivo CSV:
# df.to_csv('enlaces.csv', index=False)

# Para guardar el DataFrame en un archivo Excel:
# df.to_excel('enlaces.xlsx', index=False)

,Nombre,Enlace
0,Comisión experta,https://www.nodoxxi.cl/temas/comision-experta/
1,Consejo Constitucional,https://www.nodoxxi.cl/temas/consejo-constituc...
2,Convención,https://www.nodoxxi.cl/temas/convencion/
3,Convención Constitucional,https://www.nodoxxi.cl/temas/convencion-consti...
4,Convención Constituyente,https://www.nodoxxi.cl/temas/convencion-consti...
5,Cuidados,https://www.nodoxxi.cl/temas/cuidados/
6,Cultura,https://www.nodoxxi.cl/temas/cultura/
7,Derechos Humanos,https://www.nodoxxi.cl/temas/derechos-humanos/
8,Economía,https://www.nodoxxi.cl/temas/economia/
9,Educación,https://www.nodoxxi.cl/temas/educacion/


In [26]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
import time

# Función para crear el driver de Selenium
def create_driver(headless=False):
    options = Options()
    if headless:
        options.add_argument("--headless")
    driver = webdriver.Chrome(options=options)
    return driver

# Inicializar el WebDriver
driver = create_driver()

# Lista para almacenar los datos extraídos
all_data = []

inv2 = inv[:3].copy()

# Iterar sobre cada enlace en el DataFrame `inv`
for index, row in inv.iterrows():
    try:
        # Acceder a la página
        driver.get(row['Enlace'])
        time.sleep(1)  # Esperar a que cargue la página

        # Encontrar todos los artículos en la página
        articles = driver.find_elements(By.CSS_SELECTOR, "article.et_pb_post")

        for article in articles:
            # Extraer el título y el enlace del artículo
            try:
                title_tag = article.find_element(By.CSS_SELECTOR, "h3.entry-title a")
                title = title_tag.text.strip()
                link = title_tag.get_attribute("href")
            except:
                title = None
                link = None

            # Extraer la fecha
            try:
                date_tag = article.find_element(By.CSS_SELECTOR, "span.published")
                date = date_tag.text.strip()
            except:
                date = None

            # Extraer el autor
            try:
                author_tag = article.find_element(By.CSS_SELECTOR, "span.author a")
                author = author_tag.text.strip()
            except:
                author = None

            # Extraer el contenido del artículo
            try:
                content_tag = article.find_element(By.CSS_SELECTOR, "div.post-content-inner p")
                content = content_tag.text.strip()
            except:
                content = None

            producto = row['Nombre']

            # Guardar los datos extraídos (solo el enlace del artículo como "Enlace")
            all_data.append({
                'Producto': producto,
                'Título': title,
                'Enlace': link,
                'Fecha': date,
                'Autor': author,
                'Contenido': content
            })

    except Exception as e:
        print(f"Error al procesar {row['Enlace']}: {e}")

# Cerrar el WebDriver
driver.quit()

# Convertir los datos a un DataFrame
result_df = pd.DataFrame(all_data)

# Mostrar el resultado
result_df

# Guardar el DataFrame en un archivo CSV
# result_df.to_csv('resultados.csv', index=False)

# Guardar el DataFrame en un archivo Excel
# result_df.to_excel('resultados.xlsx', index=False)

,Producto,Título,Enlace,Fecha,Autor,Contenido
0,Comisión experta,,https://www.nodoxxi.cl/noticias/2023/a-cuatro-...,,None,
1,Comisión experta,None,None,None,,
2,Comisión experta,None,None,None,,
3,Comisión experta,None,None,None,,
4,Comisión experta,,https://www.nodoxxi.cl/portada/2024/nodo-xxi-r...,,,
...,...,...,...,...,...,...
351,Trabajo,El costo de la pandemia en su mayor medida la ...,https://www.nodoxxi.cl/noticias/2020/el-costo-...,13 Ago 2020,Fundación Nodo XXI,“Buena parte de los costos de la pandemia han ...
352,Trabajo,Defender el trabajo es salvar vidas,https://www.nodoxxi.cl/noticias/opinion/2020/d...,1 May 2020,Fundación Nodo XXI,Publica Radio u. de Chile: «urge diseñar una n...
353,Trabajo,"Trabajo, Cuidados y Violencia de Género en los...",https://www.nodoxxi.cl/publicaciones/trabajo-c...,8 Mar 2021,Camila Miranda y Giorgio Boccardo,"Durante el 2020, la crisis sanitaria y económi..."
354,Trabajo,Trabajar en Tiempos de Pandemia en Chile,https://www.nodoxxi.cl/publicaciones/trabajar-...,1 Ago 2020,Giorgio Boccardo,El informe Trabajar en tiempos de Pandemia. Tr...


# Actividades

Contenido en noticias

# Publicaciones

In [15]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
import time

def create_driver(headless=False):
    options = Options()
    if headless:
        options.add_argument("--headless")
    driver = webdriver.Chrome(options=options)
    return driver

# Inicializar el WebDriver
driver = create_driver()

# URL base
driver.get("https://www.nodoxxi.cl/publicaciones-portada/")
time.sleep(3)  # Esperar a que cargue la página

# Lista para almacenar los artículos
all_articles = []

# Encontrar el contenedor principal de los artículos
try:
    container = driver.find_element(By.CSS_SELECTOR, "div.et_pb_portfolio_items_wrapper.clearfix")
    articles = container.find_elements(By.CSS_SELECTOR, "div.et_pb_portfolio_item")
    
    for article in articles:
        # Extraer título y enlace
        try:
            title_tag = article.find_element(By.CSS_SELECTOR, "h2.et_pb_module_header a")
            title = title_tag.text.strip()
            link = title_tag.get_attribute("href")
        except:
            title = None
            link = None
        
        # Extraer categoría (Producto)
        try:
            category_tag = article.find_element(By.CSS_SELECTOR, "p.post-meta a")
            category = category_tag.text.strip()
        except:
            category = None
        
        # Extraer la imagen (opcional)
        try:
            image_tag = article.find_element(By.CSS_SELECTOR, "span.et_portfolio_image img")
            image_url = image_tag.get_attribute("src")
        except:
            image_url = None
        
        # Guardar el artículo
        all_articles.append({
            "Título": title,
            "Enlace": link,
            "Categoría": category,
            "Imagen": image_url
        })

except Exception as e:
    print(f"Error al extraer artículos: {e}")

# Cerrar el WebDriver
driver.quit()

# Convertir a DataFrame
n21_pub = pd.DataFrame(all_articles)

# Mostrar resultados
n21_pub

# Guardar el DataFrame en un archivo CSV
# n21_pub.to_csv('publicaciones.csv', index=False)

# Guardar el DataFrame en un archivo Excel
# n21_pub.to_excel('publicaciones.xlsx', index=False)

,Título,Enlace,Categoría,Imagen
0,Los 7 pilares de la Constitución neoliberal Lo...,https://www.nodoxxi.cl/publicaciones/los-7-pil...,Documentos,https://www.nodoxxi.cl/wp-content/uploads/2022...
1,Derechos sociales para una Nueva Constitución....,https://www.nodoxxi.cl/publicaciones/derechos-...,Documentos,https://www.nodoxxi.cl/wp-content/uploads/2022...
2,Orígenes socioeconómicos y trayectorias políti...,https://www.nodoxxi.cl/publicaciones/origenes-...,Documentos,https://www.nodoxxi.cl/wp-content/uploads/2021...
3,CLAVES PARA DEMOCRATIZAR LA POLÍTICA ECONÓMICA...,https://www.nodoxxi.cl/publicaciones/claves-pa...,Documentos,https://www.nodoxxi.cl/wp-content/uploads/2021...
4,Derecho a la educación y libertad de enseñanza...,https://www.nodoxxi.cl/publicaciones/derecho-a...,Documentos,https://www.nodoxxi.cl/wp-content/uploads/2021...
5,Los Cuidados al centro de la Nueva Constitución,https://www.nodoxxi.cl/publicaciones/los-cuida...,Documentos,https://www.nodoxxi.cl/wp-content/uploads/2021...
6,SERVICIOS PÚBLICOS DE CALIDAD: IDEAS PARA UNA ...,https://www.nodoxxi.cl/publicaciones/servicios...,Documentos,https://www.nodoxxi.cl/wp-content/uploads/2021...
7,Desarrollo Económico en Chile: Elementos Const...,https://www.nodoxxi.cl/publicaciones/desarroll...,Documentos,https://www.nodoxxi.cl/wp-content/uploads/2021...
8,"Trabajo, Cuidados y Violencia de Género en los...",https://www.nodoxxi.cl/publicaciones/trabajo-c...,Documentos,https://www.nodoxxi.cl/wp-content/uploads/2021...
9,Principios para una refundación orgánica del E...,https://www.nodoxxi.cl/publicaciones/principio...,Documentos,https://www.nodoxxi.cl/wp-content/uploads/2021...


# Iniciativas

In [16]:
import pandas as pd
from bs4 import BeautifulSoup

html = """
<p><strong><a href="https://www.nodoxxi.cl/iniciativas/">Iniciativas</a> /</strong> <a href="https://www.nodoxxi.cl/iniciativas/dialogos-feministas/">Diálogos Feministas</a> / <a href="https://www.nodoxxi.cl/iniciativas/compromiso-por-la-educacion/">Compromiso por la Educación</a> /<a href="https://www.nodoxxi.cl/iniciativas/espacio-coyuntura/"> Espacio de coyuntura</a>/ <a href="https://www.nodoxxi.cl/iniciativas/mesa-educacion/">Mesa de educación</a> / <a href="https://www.nodoxxi.cl/iniciativas/futuro-trabajo/">Futuro del trabajo</a> / <a href="https://www.nodoxxi.cl/iniciativas/violencia-estructural/">Mesa de violencia estructural</a></p>
"""

soup = BeautifulSoup(html, 'html.parser')

data = []

# Encuentra todos los enlaces <a> dentro del <p>
for a_tag in soup.find_all('a'):
    href = a_tag['href']
    nombre = a_tag.text.strip()  # Elimina espacios en blanco alrededor del texto
    data.append({'Nombre': nombre, 'Enlace': href})

inic = pd.DataFrame(data)
inic

# Para guardar el DataFrame en un archivo CSV:
# inv.to_csv('enlaces.csv', index=False)

# Para guardar el DataFrame en un archivo Excel:
# inv.to_excel('enlaces.xlsx', index=False)

,Nombre,Enlace
0,Iniciativas,https://www.nodoxxi.cl/iniciativas/
1,Diálogos Feministas,https://www.nodoxxi.cl/iniciativas/dialogos-fe...
2,Compromiso por la Educación,https://www.nodoxxi.cl/iniciativas/compromiso-...
3,Espacio de coyuntura,https://www.nodoxxi.cl/iniciativas/espacio-coy...
4,Mesa de educación,https://www.nodoxxi.cl/iniciativas/mesa-educac...
5,Futuro del trabajo,https://www.nodoxxi.cl/iniciativas/futuro-trab...
6,Mesa de violencia estructural,https://www.nodoxxi.cl/iniciativas/violencia-e...


In [28]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
import time

def create_driver(headless=False):
    options = Options()
    if headless:
        options.add_argument("--headless")
    driver = webdriver.Chrome(options=options)
    return driver

# Inicializar el WebDriver
driver = create_driver()

# Lista para almacenar los detalles de los artículos en cada enlace
detailed_articles = []

inic2 = inic[1:].copy()

# Iterar sobre cada enlace en el DataFrame
for index, row in inic2.iterrows():
    link = row['Enlace']
    if link:
        driver.get(link)
        time.sleep(2)  # Esperar a que cargue la página
        
        # Extraer los artículos en la página actual
        try:
            articles = driver.find_elements(By.CSS_SELECTOR, "article.et_pb_post")
            for article in articles:
                # Extraer título y enlace
                try:
                    title_tag = article.find_element(By.CSS_SELECTOR, "h2.entry-title a")
                    title = title_tag.text.strip()
                    article_link = title_tag.get_attribute("href")
                except:
                    title = None
                    article_link = None
                
                # Extraer metadatos (autor, fecha, categoría)
                try:
                    meta_tag = article.find_element(By.CSS_SELECTOR, "p.post-meta")
                    meta_text = meta_tag.text.strip()
                    # Separar autor, fecha y categorías
                    meta_parts = meta_text.split('|')
                    author = meta_parts[0].replace('por', '').strip()
                    date = meta_parts[1].strip()
                    categories = meta_parts[2].strip()
                except:
                    author = None
                    date = None
                    categories = None
                
                # Guardar el artículo detallado
                detailed_articles.append({
                    "Título": title,
                    "Enlace": article_link,
                    "Autor": author,
                    "Fecha": date,
                    "Categorías": categories
                })
        except Exception as e:
            print(f"Error al extraer artículos en {link}: {e}")

# Cerrar el WebDriver
driver.quit()

# Convertir a DataFrame
detailed_articles_df = pd.DataFrame(detailed_articles)

# Mostrar resultados
detailed_articles_df

# Guardar el DataFrame en un archivo CSV
# detailed_articles_df.to_csv('publicaciones_detalladas.csv', index=False)

# Guardar el DataFrame en un archivo Excel
# detailed_articles_df.to_excel('publicaciones_detalladas.xlsx', index=False)

,Título,Enlace,Autor,Fecha,Categorías
0,None,None,None,None,None
1,None,None,None,None,None
2,None,None,None,None,None
3,None,None,None,None,None
4,None,None,None,None,None
...,...,...,...,...,...
87,Dra. Danae Sinclaire de Nodo XXI expone en jor...,https://www.nodoxxi.cl/otros/2024/dra-danae-si...,Pierina Ferretti,15 Ago 2024,"Noticias, Otros"
88,Nodo XXI realiza un estudio sobre la eficienci...,https://www.nodoxxi.cl/portada/2024/nodo-xxi-r...,Pierina Ferretti,16 Dic 2024,"Otros, Portada"
89,Los seis nombres que se reunirán a debatir sob...,https://www.nodoxxi.cl/noticias/2023/los-seis-...,Pierina Ferretti,18 Ene 2023,"Noticias, Portada"
90,Nodo XXI realiza estudio que indaga en la rela...,https://www.nodoxxi.cl/portada/2024/nodo-xxi-r...,Pierina Ferretti,15 Dic 2024,"Noticias, Otros, Portada"


In [29]:
# Eliminar filas con NA en la columna "Enlace"
detailed_articles_df_cleaned = detailed_articles_df.dropna(subset=['Enlace'])

# Mostrar el DataFrame resultante
detailed_articles_df_cleaned

,Título,Enlace,Autor,Fecha,Categorías
10,,https://www.nodoxxi.cl/noticias/opinion/2023/e...,None,None,None
11,Después del “mayo feminista”: a profundizar la...,https://www.nodoxxi.cl/otros/2018/despues-del-...,Fundación Nodo XXI,13 Dic 2018,"Diálogos feministas, Otros"
12,Se realiza la primera versión del encuentro Di...,https://www.nodoxxi.cl/noticias/2018/se-realiz...,Fundación Nodo XXI,10 Abr 2018,"Actividades, Diálogos feministas, Noticias"
13,Se realiza la Segunda Sesión de los Diálogos F...,https://www.nodoxxi.cl/noticias/2018/se-realiz...,Fundación Nodo XXI,12 Jun 2018,"Diálogos feministas, Noticias"
14,Nodo XXI lanza infografía con perspectiva crít...,https://www.nodoxxi.cl/noticias/2018/nodo-xxi-...,Fundación Nodo XXI,24 May 2018,"Diálogos feministas, Noticias, Opinión"
25,,https://www.nodoxxi.cl/noticias/opinion/2023/e...,None,None,None
26,Agenda Corta de Fin a los abusos por estudiar ...,https://www.nodoxxi.cl/noticias/2019/agenda-co...,,23 Oct 2019,"Actividades, Compromiso, Iniciativas, Investig..."
37,,https://www.nodoxxi.cl/noticias/opinion/2023/e...,None,None,None
38,"Manifiesto Aprobar es Dignidad, Convención Con...",https://www.nodoxxi.cl/iniciativas/2020/manifi...,Fundación Nodo XXI,25 Ago 2020,"Espacio de coyuntura, Iniciativas"
39,Reflexiones en tiempos de pandemia. Análisis d...,https://www.nodoxxi.cl/noticias/2020/reflexion...,Fundación Nodo XXI,4 May 2020,"Actividades, Espacio de coyuntura, Noticias"


In [30]:
# Eliminar duplicados exactos en todas las columnas
detailed_articles_df_no_duplicates = detailed_articles_df_cleaned.drop_duplicates()

# Mostrar el DataFrame resultante
detailed_articles_df_no_duplicates

,Título,Enlace,Autor,Fecha,Categorías
10,,https://www.nodoxxi.cl/noticias/opinion/2023/e...,None,None,None
11,Después del “mayo feminista”: a profundizar la...,https://www.nodoxxi.cl/otros/2018/despues-del-...,Fundación Nodo XXI,13 Dic 2018,"Diálogos feministas, Otros"
12,Se realiza la primera versión del encuentro Di...,https://www.nodoxxi.cl/noticias/2018/se-realiz...,Fundación Nodo XXI,10 Abr 2018,"Actividades, Diálogos feministas, Noticias"
13,Se realiza la Segunda Sesión de los Diálogos F...,https://www.nodoxxi.cl/noticias/2018/se-realiz...,Fundación Nodo XXI,12 Jun 2018,"Diálogos feministas, Noticias"
14,Nodo XXI lanza infografía con perspectiva crít...,https://www.nodoxxi.cl/noticias/2018/nodo-xxi-...,Fundación Nodo XXI,24 May 2018,"Diálogos feministas, Noticias, Opinión"
26,Agenda Corta de Fin a los abusos por estudiar ...,https://www.nodoxxi.cl/noticias/2019/agenda-co...,,23 Oct 2019,"Actividades, Compromiso, Iniciativas, Investig..."
38,"Manifiesto Aprobar es Dignidad, Convención Con...",https://www.nodoxxi.cl/iniciativas/2020/manifi...,Fundación Nodo XXI,25 Ago 2020,"Espacio de coyuntura, Iniciativas"
39,Reflexiones en tiempos de pandemia. Análisis d...,https://www.nodoxxi.cl/noticias/2020/reflexion...,Fundación Nodo XXI,4 May 2020,"Actividades, Espacio de coyuntura, Noticias"
40,Reflexiones políticas en tiempos de pandemia.,https://www.nodoxxi.cl/iniciativas/2020/reflex...,Fundación Nodo XXI,18 Jul 2020,"Espacio de coyuntura, Iniciativas"
41,Mesa de Coyuntura de Nodo XXI evalúa el Acuerd...,https://www.nodoxxi.cl/noticias/2019/mesa-de-c...,Fundación Nodo XXI,21 Nov 2019,"Actividades, Espacio de coyuntura, Noticias"


# Formación

Contiene videos

# Corpus

In [51]:
import pandas as pd

# Lista de DataFrames
dfs = [n21, result_df, n21_pub, detailed_articles_df_no_duplicates]

# Concatenar todos los DataFrames en uno solo
df_n21 = pd.concat(dfs, ignore_index=True)

# Mostrar el DataFrame resultante
df_n21

,Título,Enlace,Fecha,Producto,Autor,Contenido,Categoría,Imagen,Categorías
0,"A cuatro años de la revuelta social, volver a ...",https://www.nodoxxi.cl/noticias/2023/a-cuatro-...,18 Oct 2023,"Estado Social, Política, Sociedad",NaN,NaN,NaN,NaN,NaN
1,"El país no necesita al CAE, el CAE necesita so...",https://www.nodoxxi.cl/noticias/opinion/2023/e...,7 Oct 2023,Educación,NaN,NaN,NaN,NaN,NaN
2,Conmemoración 50 años del Golpe de Estado: ¿Ha...,https://www.nodoxxi.cl/noticias/2023/conmemora...,23 Jul 2023,"Derechos Humanos, Política",NaN,NaN,NaN,NaN,NaN
3,Presentación de la IPN Nº 10.887,https://www.nodoxxi.cl/noticias/2023/presentac...,23 Jul 2023,"Consejo Constitucional, Estado Social",NaN,NaN,NaN,NaN,NaN
4,DECLARACIÓN INICIATIVAS POPULARES CIUDADANAS,https://www.nodoxxi.cl/noticias/2023/declaraci...,11 Jul 2023,Consejo Constitucional,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
523,Dra. Danae Sinclaire de Nodo XXI expone en jor...,https://www.nodoxxi.cl/otros/2024/dra-danae-si...,15 Ago 2024,NaN,Pierina Ferretti,NaN,NaN,NaN,"Noticias, Otros"
524,Nodo XXI realiza un estudio sobre la eficienci...,https://www.nodoxxi.cl/portada/2024/nodo-xxi-r...,16 Dic 2024,NaN,Pierina Ferretti,NaN,NaN,NaN,"Otros, Portada"
525,Los seis nombres que se reunirán a debatir sob...,https://www.nodoxxi.cl/noticias/2023/los-seis-...,18 Ene 2023,NaN,Pierina Ferretti,NaN,NaN,NaN,"Noticias, Portada"
526,Nodo XXI realiza estudio que indaga en la rela...,https://www.nodoxxi.cl/portada/2024/nodo-xxi-r...,15 Dic 2024,NaN,Pierina Ferretti,NaN,NaN,NaN,"Noticias, Otros, Portada"


In [52]:
import pandas as pd

# Supongamos que df_n21 es tu DataFrame combinado
# Eliminar filas duplicadas exactas, manteniendo la primera entrada
df_n21 = df_n21.drop_duplicates()

# Combinar las columnas "Producto", "Categoría" y "Categorías" en una sola columna "Producto"
df_n21['Producto'] = df_n21[['Producto', 'Categoría', 'Categorías']].fillna('').agg(' '.join, axis=1).str.strip()

# Eliminar las columnas originales "Producto", "Categoría" y "Categorías" si ya no las necesitas
df_n21 = df_n21.drop(columns=['Categoría', 'Categorías'])

C:\Users\diego\AppData\Local\Temp\ipykernel_8248\3471415697.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_n21['Producto'] = df_n21[['Producto', 'Categoría', 'Categorías']].fillna('').agg(' '.join, axis=1).str.strip()


In [53]:
df_n21

,Título,Enlace,Fecha,Producto,Autor,Contenido,Imagen
0,"A cuatro años de la revuelta social, volver a ...",https://www.nodoxxi.cl/noticias/2023/a-cuatro-...,18 Oct 2023,"Estado Social, Política, Sociedad",NaN,NaN,NaN
1,"El país no necesita al CAE, el CAE necesita so...",https://www.nodoxxi.cl/noticias/opinion/2023/e...,7 Oct 2023,Educación,NaN,NaN,NaN
2,Conmemoración 50 años del Golpe de Estado: ¿Ha...,https://www.nodoxxi.cl/noticias/2023/conmemora...,23 Jul 2023,"Derechos Humanos, Política",NaN,NaN,NaN
3,Presentación de la IPN Nº 10.887,https://www.nodoxxi.cl/noticias/2023/presentac...,23 Jul 2023,"Consejo Constitucional, Estado Social",NaN,NaN,NaN
4,DECLARACIÓN INICIATIVAS POPULARES CIUDADANAS,https://www.nodoxxi.cl/noticias/2023/declaraci...,11 Jul 2023,Consejo Constitucional,NaN,NaN,NaN
...,...,...,...,...,...,...,...
523,Dra. Danae Sinclaire de Nodo XXI expone en jor...,https://www.nodoxxi.cl/otros/2024/dra-danae-si...,15 Ago 2024,"Noticias, Otros",Pierina Ferretti,NaN,NaN
524,Nodo XXI realiza un estudio sobre la eficienci...,https://www.nodoxxi.cl/portada/2024/nodo-xxi-r...,16 Dic 2024,"Otros, Portada",Pierina Ferretti,NaN,NaN
525,Los seis nombres que se reunirán a debatir sob...,https://www.nodoxxi.cl/noticias/2023/los-seis-...,18 Ene 2023,"Noticias, Portada",Pierina Ferretti,NaN,NaN
526,Nodo XXI realiza estudio que indaga en la rela...,https://www.nodoxxi.cl/portada/2024/nodo-xxi-r...,15 Dic 2024,"Noticias, Otros, Portada",Pierina Ferretti,NaN,NaN


In [57]:
df_n21.Fecha.unique()

array(['18 Oct 2023', '7 Oct 2023', '23 Jul 2023', '11 Jul 2023',
       '5 Jul 2023', '4 Jul 2023', '', None, '7 Jul 2023', '20 May 2023',
       '7 Dic 2023', '6 May 2023', '3 Nov 2021', '13 Oct 2021',
       '5 Jul 2022', '20 Ene 2022', '5 Ene 2022', '2 Nov 2021',
       '22 Jun 2022', '18 Ene 2022', '4 Ene 2022', '24 Ene 2022',
       '9 Ene 2022', '5 Nov 2021', '15 Oct 2021', '21 Sep 2021',
       '27 Oct 2021', '9 Oct 2021', '13 Sep 2021', '20 Oct 2021',
       '7 Oct 2021', '22 Jul 2022', '12 Abr 2021', '21 Jun 2020',
       '4 Mar 2020', '5 Oct 2022', '11 Sep 2021', '10 Feb 2019',
       '18 Oct 2022', '1 Jul 2022', '1 Oct 2018', '11 Jul 2022',
       '25 Mar 2021', '1 May 2020', '24 Mar 2020', '28 Dic 2021',
       '8 Ene 2021', '8 Abr 2020', '19 Abr 2021', '1 Abr 2020',
       '26 Ago 2020', '27 Jun 2023', '1 Jul 2021', '31 Dic 2022',
       '1 Dic 2020', '22 May 2023', '3 Oct 2021', '4 Ago 2020',
       '25 Nov 2021', '8 Mar 2021', '28 Oct 2020', '6 Mar 2020',
       '7 Mar 

In [58]:
import pandas as pd

# Diccionario de mapeo de meses en español a números
meses_espanol_a_numero = {
    'Ene': 1, 'Feb': 2, 'Mar': 3, 'Abr': 4, 'May': 5, 'Jun': 6,
    'Jul': 7, 'Ago': 8, 'Sep': 9, 'Oct': 10, 'Nov': 11, 'Dic': 12
}

# Función para convertir la columna 'Fecha' en 'Fecha de publicación'
def convertir_fecha(fecha):
    if not isinstance(fecha, str) or fecha.strip() == '':
        return pd.NaT  # Maneja valores nulos o vacíos
    
    partes = fecha.split()
    if len(partes) != 3:
        return pd.NaT  # Si el formato no es "día mes año", retorna NaT
    
    dia, mes, anio = partes
    mes = meses_espanol_a_numero.get(mes)  # Convertir mes a número
    
    if not mes:
        return pd.NaT  # Si el mes no está en el diccionario, retornar NaT
    
    return pd.to_datetime(f"{anio}-{mes}-{dia}", format='%Y-%m-%d')

# Aplicar la función a la columna 'Fecha'
df_n21['Fecha de publicación'] = df_n21['Fecha'].apply(convertir_fecha)
df_n21

,Título,Enlace,Fecha,Producto,Autor,Contenido,Imagen,Fecha de publicación
0,"A cuatro años de la revuelta social, volver a ...",https://www.nodoxxi.cl/noticias/2023/a-cuatro-...,18 Oct 2023,"Estado Social, Política, Sociedad",NaN,NaN,NaN,2023-10-18
1,"El país no necesita al CAE, el CAE necesita so...",https://www.nodoxxi.cl/noticias/opinion/2023/e...,7 Oct 2023,Educación,NaN,NaN,NaN,2023-10-07
2,Conmemoración 50 años del Golpe de Estado: ¿Ha...,https://www.nodoxxi.cl/noticias/2023/conmemora...,23 Jul 2023,"Derechos Humanos, Política",NaN,NaN,NaN,2023-07-23
3,Presentación de la IPN Nº 10.887,https://www.nodoxxi.cl/noticias/2023/presentac...,23 Jul 2023,"Consejo Constitucional, Estado Social",NaN,NaN,NaN,2023-07-23
4,DECLARACIÓN INICIATIVAS POPULARES CIUDADANAS,https://www.nodoxxi.cl/noticias/2023/declaraci...,11 Jul 2023,Consejo Constitucional,NaN,NaN,NaN,2023-07-11
...,...,...,...,...,...,...,...,...
523,Dra. Danae Sinclaire de Nodo XXI expone en jor...,https://www.nodoxxi.cl/otros/2024/dra-danae-si...,15 Ago 2024,"Noticias, Otros",Pierina Ferretti,NaN,NaN,2024-08-15
524,Nodo XXI realiza un estudio sobre la eficienci...,https://www.nodoxxi.cl/portada/2024/nodo-xxi-r...,16 Dic 2024,"Otros, Portada",Pierina Ferretti,NaN,NaN,2024-12-16
525,Los seis nombres que se reunirán a debatir sob...,https://www.nodoxxi.cl/noticias/2023/los-seis-...,18 Ene 2023,"Noticias, Portada",Pierina Ferretti,NaN,NaN,2023-01-18
526,Nodo XXI realiza estudio que indaga en la rela...,https://www.nodoxxi.cl/portada/2024/nodo-xxi-r...,15 Dic 2024,"Noticias, Otros, Portada",Pierina Ferretti,NaN,NaN,2024-12-15


In [65]:
import pandas as pd
import numpy as np
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
import time

def create_driver(headless=False):
    options = Options()
    if headless:
        options.add_argument("--headless")
    driver = webdriver.Chrome(options=options)
    return driver

driver = create_driver()

def extraer_corpus_selenium(url):

    try:
        driver.get(url)
        time.sleep(1)  # Esperar a que cargue la página
        
        # Extraer el texto de todos los párrafos <p>
        parrafos = driver.find_elements(By.TAG_NAME, 'p')
        corpus = ' '.join([p.text for p in parrafos])
        
        return corpus if corpus.strip() else None
    except Exception as e:
        print(f"Error al acceder a {url}: {e}")
        return None

def actualizar_corpus_selenium(df):
    for index, row in df.iterrows():
        if pd.isna(row['Corpus']):  # Solo procesar filas con "Corpus" vacío
            enlace = row['Enlace']
            
            # Verificar si ya se ha extraído el Corpus para este enlace
            corpus_existente = df.loc[df['Enlace'] == enlace, 'Corpus'].dropna()
            if not corpus_existente.empty:
                df.at[index, 'Corpus'] = corpus_existente.iloc[0]
            else:
                corpus = extraer_corpus_selenium(enlace)
                if corpus:
                    df.at[index, 'Corpus'] = corpus
                    # Actualizar todas las filas con el mismo enlace
                    df.loc[df['Enlace'] == enlace, 'Corpus'] = corpus
    return df

# Prueba con un subconjunto del DataFrame
av = df_n21.copy()
av["Corpus"] = np.nan
aver = actualizar_corpus_selenium(av)

driver.quit()

aver

Error al acceder a https://www.nodoxxi.cl/noticias/2019/nodo-xxi-presenta-propuesta-de-condonacion-de-deudas-por-estudiar/: Message: stale element reference: stale element not found in the current frame
  (Session info: chrome=132.0.6834.159); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#stale-element-reference-exception
Stacktrace:
	GetHandleVerifier [0x00007FF6192C02F5+28725]
	(No symbol) [0x00007FF619222AE0]
	(No symbol) [0x00007FF6190B510A]
	(No symbol) [0x00007FF6190BBD70]
	(No symbol) [0x00007FF6190BEABC]
	(No symbol) [0x00007FF6190BEB8F]
	(No symbol) [0x00007FF61910313F]
	(No symbol) [0x00007FF61912FFAA]
	(No symbol) [0x00007FF6190FBF86]
	(No symbol) [0x00007FF6191301C0]
	(No symbol) [0x00007FF619150181]
	(No symbol) [0x00007FF61912FD53]
	(No symbol) [0x00007FF6190FA0E3]
	(No symbol) [0x00007FF6190FB471]
	GetHandleVerifier [0x00007FF6195EF30D+3366989]
	GetHandleVerifier [0x00007FF6196012F0+3440688]
	GetHan

,Título,Enlace,Fecha,Producto,Autor,Contenido,Imagen,Fecha de publicación,Corpus
0,"A cuatro años de la revuelta social, volver a ...",https://www.nodoxxi.cl/noticias/2023/a-cuatro-...,18 Oct 2023,"Estado Social, Política, Sociedad",NaN,NaN,NaN,2023-10-18,A cuatro años...
1,"El país no necesita al CAE, el CAE necesita so...",https://www.nodoxxi.cl/noticias/opinion/2023/e...,7 Oct 2023,Educación,NaN,NaN,NaN,2023-10-07,En entrevista...
2,Conmemoración 50 años del Golpe de Estado: ¿Ha...,https://www.nodoxxi.cl/noticias/2023/conmemora...,23 Jul 2023,"Derechos Humanos, Política",NaN,NaN,NaN,2023-07-23,Hace diez año...
3,Presentación de la IPN Nº 10.887,https://www.nodoxxi.cl/noticias/2023/presentac...,23 Jul 2023,"Consejo Constitucional, Estado Social",NaN,NaN,NaN,2023-07-23,El miércoles ...
4,DECLARACIÓN INICIATIVAS POPULARES CIUDADANAS,https://www.nodoxxi.cl/noticias/2023/declaraci...,11 Jul 2023,Consejo Constitucional,NaN,NaN,NaN,2023-07-11,Estamos acá r...
...,...,...,...,...,...,...,...,...,...
523,Dra. Danae Sinclaire de Nodo XXI expone en jor...,https://www.nodoxxi.cl/otros/2024/dra-danae-si...,15 Ago 2024,"Noticias, Otros",Pierina Ferretti,NaN,NaN,2024-08-15,El pasado jue...
524,Nodo XXI realiza un estudio sobre la eficienci...,https://www.nodoxxi.cl/portada/2024/nodo-xxi-r...,16 Dic 2024,"Otros, Portada",Pierina Ferretti,NaN,NaN,2024-12-16,En el marco d...
525,Los seis nombres que se reunirán a debatir sob...,https://www.nodoxxi.cl/noticias/2023/los-seis-...,18 Ene 2023,"Noticias, Portada",Pierina Ferretti,NaN,NaN,2023-01-18,La Fundación ...
526,Nodo XXI realiza estudio que indaga en la rela...,https://www.nodoxxi.cl/portada/2024/nodo-xxi-r...,15 Dic 2024,"Noticias, Otros, Portada",Pierina Ferretti,NaN,NaN,2024-12-15,Ganar sin per...


In [66]:
import pandas as pd

# Asumiendo que tu DataFrame se llama "aver"
# y la columna que quieres verificar se llama "Enlace"

# 1. Identificar las filas con "Enlace" vacío
filas_vacias = aver['Enlace'].isnull() | (aver['Enlace'] == '')

# 2. Invertir la condición para obtener las filas que NO están vacías
filas_no_vacias = ~filas_vacias

# 3. Crear un nuevo DataFrame con solo las filas no vacías
aver_sin_enlaces_vacios = aver[filas_no_vacias]

# 4. (Opcional) Si quieres modificar el DataFrame original en lugar de crear uno nuevo:
# aver.drop(aver[filas_vacias].index, inplace=True)

# Mostrar el DataFrame resultante (o el original si usaste la opción inplace)
aver_sin_enlaces_vacios

,Título,Enlace,Fecha,Producto,Autor,Contenido,Imagen,Fecha de publicación,Corpus
0,"A cuatro años de la revuelta social, volver a ...",https://www.nodoxxi.cl/noticias/2023/a-cuatro-...,18 Oct 2023,"Estado Social, Política, Sociedad",NaN,NaN,NaN,2023-10-18,A cuatro años...
1,"El país no necesita al CAE, el CAE necesita so...",https://www.nodoxxi.cl/noticias/opinion/2023/e...,7 Oct 2023,Educación,NaN,NaN,NaN,2023-10-07,En entrevista...
2,Conmemoración 50 años del Golpe de Estado: ¿Ha...,https://www.nodoxxi.cl/noticias/2023/conmemora...,23 Jul 2023,"Derechos Humanos, Política",NaN,NaN,NaN,2023-07-23,Hace diez año...
3,Presentación de la IPN Nº 10.887,https://www.nodoxxi.cl/noticias/2023/presentac...,23 Jul 2023,"Consejo Constitucional, Estado Social",NaN,NaN,NaN,2023-07-23,El miércoles ...
4,DECLARACIÓN INICIATIVAS POPULARES CIUDADANAS,https://www.nodoxxi.cl/noticias/2023/declaraci...,11 Jul 2023,Consejo Constitucional,NaN,NaN,NaN,2023-07-11,Estamos acá r...
...,...,...,...,...,...,...,...,...,...
523,Dra. Danae Sinclaire de Nodo XXI expone en jor...,https://www.nodoxxi.cl/otros/2024/dra-danae-si...,15 Ago 2024,"Noticias, Otros",Pierina Ferretti,NaN,NaN,2024-08-15,El pasado jue...
524,Nodo XXI realiza un estudio sobre la eficienci...,https://www.nodoxxi.cl/portada/2024/nodo-xxi-r...,16 Dic 2024,"Otros, Portada",Pierina Ferretti,NaN,NaN,2024-12-16,En el marco d...
525,Los seis nombres que se reunirán a debatir sob...,https://www.nodoxxi.cl/noticias/2023/los-seis-...,18 Ene 2023,"Noticias, Portada",Pierina Ferretti,NaN,NaN,2023-01-18,La Fundación ...
526,Nodo XXI realiza estudio que indaga en la rela...,https://www.nodoxxi.cl/portada/2024/nodo-xxi-r...,15 Dic 2024,"Noticias, Otros, Portada",Pierina Ferretti,NaN,NaN,2024-12-15,Ganar sin per...


In [69]:
aver["Título"].unique().tolist()

['A cuatro años de la revuelta social, volver a las causas',
 'El país no necesita al CAE, el CAE necesita solución',
 'Conmemoración 50 años del Golpe de Estado: ¿Hay derecha democrática?',
 'Presentación de la IPN Nº 10.887',
 'DECLARACIÓN INICIATIVAS POPULARES CIUDADANAS',
 'Modernización del Estado y Sistema de financiamiento de la Educación Superior',
 'Entrevista a Felipe Ruiz: nuevas formas de trabajo, las nuevas tecnologías y los desafíos de la juventud sindicalista en la región.',
 'Nunca Más. A 50 años del Golpe de Estado',
 'No son péndulo, son oposicionistas',
 'Reactivación educativa para reimaginar la educación',
 '',
 None,
 'PROHIBICIÓN DEL LUCRO CON LOS DERECHOS',
 'Análisis feministas del anteproyecto constitucional 2023',
 'Retrocesos en el texto propuesto por el Consejo Constitucional',
 'La pelota en la cancha republicana',
 'En la elección de mañana hay mucho en juego',
 'Los Cuidados al centro de la Nueva Constitución',
 'Daniela López Leiva, socia del estudio ju

In [71]:
import pandas as pd
import numpy as np
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
import time

def create_driver(headless=False):
    options = Options()
    if headless:
        options.add_argument("--headless")
    driver = webdriver.Chrome(options=options)
    return driver

driver = create_driver()

def extraer_metadata_selenium(url):
    try:
        driver.get(url)
        time.sleep(1)  # Esperar a que cargue la página
        
        # Extraer el título
        titulo_element = driver.find_element(By.CSS_SELECTOR, '.et_pb_text_1_tb_body .et_pb_text_inner')
        titulo = titulo_element.text if titulo_element else None
        
        # Extraer la fecha
        fecha_element = driver.find_element(By.CSS_SELECTOR, '.et_pb_text_3_tb_body .et_pb_text_inner')
        fecha = fecha_element.text if fecha_element else None
        
        # Extraer el autor
        autor_element = driver.find_element(By.CSS_SELECTOR, '.et_pb_text_2_tb_body .et_pb_text_inner')
        autor = autor_element.text if autor_element else None
        
        # Extraer el producto
        producto_element = driver.find_element(By.CSS_SELECTOR, '.et_pb_text_0_tb_body .et_pb_text_inner')
        producto = producto_element.text if producto_element else None
        
        return {
            'Título': titulo,
            'Fecha': fecha,
            'Autor': autor,
            'Producto': producto
        }
    except Exception as e:
        print(f"Error al acceder a {url}: {e}")
        return {
            'Título': None,
            'Fecha': None,
            'Autor': None,
            'Producto': None
        }

def actualizar_metadata_selenium(df):
    for index, row in df.iterrows():
        if pd.isna(row['Título']) or pd.isna(row['Fecha']) or pd.isna(row['Autor']) or pd.isna(row['Producto']):  # Solo procesar filas con metadata vacía
            enlace = row['Enlace']
            
            # Verificar si ya se ha extraído la metadata para este enlace
            metadata_existente = df.loc[df['Enlace'] == enlace, ['Título', 'Fecha', 'Autor', 'Producto']].dropna()
            if not metadata_existente.empty:
                # Si existe metadata para este enlace, copiarla a la fila actual
                df.at[index, 'Título'] = metadata_existente.iloc[0]['Título']
                df.at[index, 'Fecha'] = metadata_existente.iloc[0]['Fecha']
                df.at[index, 'Autor'] = metadata_existente.iloc[0]['Autor']
                df.at[index, 'Producto'] = metadata_existente.iloc[0]['Producto']
            else:
                # Si no existe metadata, extraerla y actualizar todas las filas con el mismo enlace
                metadata = extraer_metadata_selenium(enlace)
                if metadata['Título']:
                    df.at[index, 'Título'] = metadata['Título']
                    df.at[index, 'Fecha'] = metadata['Fecha']
                    df.at[index, 'Autor'] = metadata['Autor']
                    df.at[index, 'Producto'] = metadata['Producto']
                    # Actualizar todas las filas con el mismo enlace
                    df.loc[df['Enlace'] == enlace, 'Título'] = metadata['Título']
                    df.loc[df['Enlace'] == enlace, 'Fecha'] = metadata['Fecha']
                    df.loc[df['Enlace'] == enlace, 'Autor'] = metadata['Autor']
                    df.loc[df['Enlace'] == enlace, 'Producto'] = metadata['Producto']
    return df

# Prueba con un subconjunto del DataFrame
nxxi = aver_sin_enlaces_vacios.copy()
nxxi["Título"] = np.nan
nxxi["Fecha"] = np.nan
nxxi["Autor"] = np.nan
nxxi["Producto"] = np.nan
nxxi2 = actualizar_metadata_selenium(nxxi)

driver.quit()

nxxi2

Error al acceder a https://www.nodoxxi.cl/noticias/2022/un-espacio-de-encuentro-y-debate/: Message: no such element: Unable to locate element: {"method":"css selector","selector":".et_pb_text_0_tb_body .et_pb_text_inner"}
  (Session info: chrome=132.0.6834.159); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x00007FF6192C02F5+28725]
	(No symbol) [0x00007FF619222AE0]
	(No symbol) [0x00007FF6190B510A]
	(No symbol) [0x00007FF6191093D2]
	(No symbol) [0x00007FF6191095FC]
	(No symbol) [0x00007FF619153407]
	(No symbol) [0x00007FF61912FFEF]
	(No symbol) [0x00007FF619150181]
	(No symbol) [0x00007FF61912FD53]
	(No symbol) [0x00007FF6190FA0E3]
	(No symbol) [0x00007FF6190FB471]
	GetHandleVerifier [0x00007FF6195EF30D+3366989]
	GetHandleVerifier [0x00007FF6196012F0+3440688]
	GetHandleVerifier [0x00007FF6195F78FD+3401277]
	GetHandleVerifier [0x00007FF61938AAAB+858091]
	(N

,Título,Enlace,Fecha,Producto,Autor,Contenido,Imagen,Fecha de publicación,Corpus
0,"A CUATRO AÑOS DE LA REVUELTA SOCIAL, VOLVER A ...",https://www.nodoxxi.cl/noticias/2023/a-cuatro-...,18 octubre 2023,Estado Social | Política | Sociedad,Camila Miranda,NaN,NaN,2023-10-18,A cuatro años...
1,"EL PAÍS NO NECESITA AL CAE, EL CAE NECESITA SO...",https://www.nodoxxi.cl/noticias/opinion/2023/e...,7 octubre 2023,Educación,Nodo xxi,NaN,NaN,2023-10-07,En entrevista...
2,CONMEMORACIÓN 50 AÑOS DEL GOLPE DE ESTADO: ¿HA...,https://www.nodoxxi.cl/noticias/2023/conmemora...,23 julio 2023,Derechos Humanos | Política,Camila Miranda y Pierina Ferretti,NaN,NaN,2023-07-23,Hace diez año...
3,PRESENTACIÓN DE LA IPN Nº 10.887,https://www.nodoxxi.cl/noticias/2023/presentac...,23 julio 2023,Consejo Constitucional | Estado Social,Nodo xxi,NaN,NaN,2023-07-23,El miércoles ...
4,DECLARACIÓN INICIATIVAS POPULARES CIUDAD...,https://www.nodoxxi.cl/noticias/2023/declaraci...,11 julio 2023,Consejo Constitucional,Camila Miranda,NaN,NaN,2023-07-11,Estamos acá r...
...,...,...,...,...,...,...,...,...,...
523,NaN,https://www.nodoxxi.cl/otros/2024/dra-danae-si...,NaN,NaN,NaN,NaN,NaN,2024-08-15,El pasado jue...
524,NaN,https://www.nodoxxi.cl/portada/2024/nodo-xxi-r...,NaN,NaN,NaN,NaN,NaN,2024-12-16,En el marco d...
525,NaN,https://www.nodoxxi.cl/noticias/2023/los-seis-...,NaN,NaN,NaN,NaN,NaN,2023-01-18,La Fundación ...
526,NaN,https://www.nodoxxi.cl/portada/2024/nodo-xxi-r...,NaN,NaN,NaN,NaN,NaN,2024-12-15,Ganar sin per...


In [72]:
import pandas as pd
import numpy as np
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
import time

def create_driver(headless=False):
    options = Options()
    if headless:
        options.add_argument("--headless")
    driver = webdriver.Chrome(options=options)
    return driver

driver = create_driver()

def extraer_metadata_selenium(url):
    try:
        driver.get(url)
        time.sleep(1)  # Esperar a que cargue la página
        
        # Extraer el título
        titulo_element = driver.find_element(By.CSS_SELECTOR, '.et_pb_row_0_tb_body .et_pb_text_1_tb_body .et_pb_text_inner')
        titulo = titulo_element.text if titulo_element else None
        
        # Extraer la fecha
        fecha_element = driver.find_element(By.CSS_SELECTOR, '.et_pb_column_2_tb_body .et_pb_text_3_tb_body .et_pb_text_inner')
        fecha = fecha_element.text if fecha_element else None
        
        # Extraer el autor
        autor_element = driver.find_element(By.CSS_SELECTOR, '.et_pb_column_1_tb_body .et_pb_text_2_tb_body .et_pb_text_inner')
        autor = autor_element.text if autor_element else None
        
        # Extraer el producto
        producto_element = driver.find_element(By.CSS_SELECTOR, '.et_pb_inline_icon[data-icon="9"] span')
        producto = producto_element.text if producto_element else None
        
        return {
            'Título': titulo,
            'Fecha': fecha,
            'Autor': autor,
            'Producto': producto
        }
    except Exception as e:
        print(f"Error al acceder a {url}: {e}")
        return {
            'Título': None,
            'Fecha': None,
            'Autor': None,
            'Producto': None
        }

def actualizar_metadata_selenium(df):
    for index, row in df.iterrows():
        if pd.isna(row['Título']) or pd.isna(row['Fecha']) or pd.isna(row['Autor']) or pd.isna(row['Producto']):  # Solo procesar filas con metadata vacía
            enlace = row['Enlace']
            
            # Verificar si ya se ha extraído la metadata para este enlace
            metadata_existente = df.loc[df['Enlace'] == enlace, ['Título', 'Fecha', 'Autor', 'Producto']].dropna()
            if not metadata_existente.empty:
                # Si existe metadata para este enlace, copiarla a la fila actual
                df.at[index, 'Título'] = metadata_existente.iloc[0]['Título']
                df.at[index, 'Fecha'] = metadata_existente.iloc[0]['Fecha']
                df.at[index, 'Autor'] = metadata_existente.iloc[0]['Autor']
                df.at[index, 'Producto'] = metadata_existente.iloc[0]['Producto']
            else:
                # Si no existe metadata, extraerla y actualizar todas las filas con el mismo enlace
                metadata = extraer_metadata_selenium(enlace)
                if metadata['Título']:
                    df.at[index, 'Título'] = metadata['Título']
                    df.at[index, 'Fecha'] = metadata['Fecha']
                    df.at[index, 'Autor'] = metadata['Autor']
                    df.at[index, 'Producto'] = metadata['Producto']
                    # Actualizar todas las filas con el mismo enlace
                    df.loc[df['Enlace'] == enlace, 'Título'] = metadata['Título']
                    df.loc[df['Enlace'] == enlace, 'Fecha'] = metadata['Fecha']
                    df.loc[df['Enlace'] == enlace, 'Autor'] = metadata['Autor']
                    df.loc[df['Enlace'] == enlace, 'Producto'] = metadata['Producto']
    return df

# Prueba con un subconjunto del DataFrame
nxxi22 = nxxi2.copy()
nxxi22["Título"] = np.nan
nxxi22["Fecha"] = np.nan
nxxi22["Autor"] = np.nan
nxxi22["Producto"] = np.nan
nxxi3 = actualizar_metadata_selenium(nxxi22)

driver.quit()

nxxi3

Error al acceder a https://www.nodoxxi.cl/noticias/entrevistas/2020/4742/: Message: no such element: Unable to locate element: {"method":"css selector","selector":".et_pb_row_0_tb_body .et_pb_text_1_tb_body .et_pb_text_inner"}
  (Session info: chrome=132.0.6834.159); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x00007FF6192C02F5+28725]
	(No symbol) [0x00007FF619222AE0]
	(No symbol) [0x00007FF6190B510A]
	(No symbol) [0x00007FF6191093D2]
	(No symbol) [0x00007FF6191095FC]
	(No symbol) [0x00007FF619153407]
	(No symbol) [0x00007FF61912FFEF]
	(No symbol) [0x00007FF619150181]
	(No symbol) [0x00007FF61912FD53]
	(No symbol) [0x00007FF6190FA0E3]
	(No symbol) [0x00007FF6190FB471]
	GetHandleVerifier [0x00007FF6195EF30D+3366989]
	GetHandleVerifier [0x00007FF6196012F0+3440688]
	GetHandleVerifier [0x00007FF6195F78FD+3401277]
	GetHandleVerifier [0x00007FF61938AAAB+858091

,Título,Enlace,Fecha,Producto,Autor,Contenido,Imagen,Fecha de publicación,Corpus
0,"A CUATRO AÑOS DE LA REVUELTA SOCIAL, VOLVER A ...",https://www.nodoxxi.cl/noticias/2023/a-cuatro-...,18 octubre 2023,Noticias,Camila Miranda,NaN,NaN,2023-10-18,A cuatro años...
1,"EL PAÍS NO NECESITA AL CAE, EL CAE NECESITA SO...",https://www.nodoxxi.cl/noticias/opinion/2023/e...,7 octubre 2023,Noticias,Nodo xxi,NaN,NaN,2023-10-07,En entrevista...
2,CONMEMORACIÓN 50 AÑOS DEL GOLPE DE ESTADO: ¿HA...,https://www.nodoxxi.cl/noticias/2023/conmemora...,23 julio 2023,Noticias,Camila Miranda y Pierina Ferretti,NaN,NaN,2023-07-23,Hace diez año...
3,PRESENTACIÓN DE LA IPN Nº 10.887,https://www.nodoxxi.cl/noticias/2023/presentac...,23 julio 2023,Noticias,Nodo xxi,NaN,NaN,2023-07-23,El miércoles ...
4,DECLARACIÓN INICIATIVAS POPULARES CIUDAD...,https://www.nodoxxi.cl/noticias/2023/declaraci...,11 julio 2023,Noticias,Camila Miranda,NaN,NaN,2023-07-11,Estamos acá r...
...,...,...,...,...,...,...,...,...,...
523,DRA. DANAE SINCLAIRE DE NODO XXI EXPONE EN JOR...,https://www.nodoxxi.cl/otros/2024/dra-danae-si...,15 agosto 2024,Otros,Pierina Ferretti,NaN,NaN,2024-08-15,El pasado jue...
524,NODO XXI REALIZA UN ESTUDIO SOBRE LA EFICIENCI...,https://www.nodoxxi.cl/portada/2024/nodo-xxi-r...,16 diciembre 2024,Otros,Pierina Ferretti,NaN,NaN,2024-12-16,En el marco d...
525,LOS SEIS NOMBRES QUE SE REUNIRÁN A DEBATIR SOB...,https://www.nodoxxi.cl/noticias/2023/los-seis-...,18 enero 2023,Noticias,Pierina Ferretti,NaN,NaN,2023-01-18,La Fundación ...
526,NODO XXI REALIZA ESTUDIO QUE INDAGA EN LA RELA...,https://www.nodoxxi.cl/portada/2024/nodo-xxi-r...,15 diciembre 2024,Otros,Pierina Ferretti,NaN,NaN,2024-12-15,Ganar sin per...


In [73]:
import pandas as pd
import numpy as np
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
import time

def create_driver(headless=False):
    options = Options()
    if headless:
        options.add_argument("--headless")
    driver = webdriver.Chrome(options=options)
    return driver

driver = create_driver()

def extraer_metadata_selenium(url):
    try:
        driver.get(url)
        time.sleep(1)  # Esperar a que cargue la página
        
        # Intentar extraer el título con el selector original
        try:
            titulo_element = driver.find_element(By.CSS_SELECTOR, '.et_pb_row_0_tb_body .et_pb_text_1_tb_body .et_pb_text_inner')
            titulo = titulo_element.text if titulo_element else None
        except:
            titulo = None
        
        # Si no se encuentra, intentar con el nuevo selector
        if not titulo:
            try:
                titulo_element = driver.find_element(By.CSS_SELECTOR, '.et_pb_text_1_tb_body .et_pb_text_inner')
                titulo = titulo_element.text if titulo_element else None
            except:
                titulo = None

        # Extraer la fecha (mantiene el mismo selector)
        try:
            fecha_element = driver.find_element(By.CSS_SELECTOR, '.et_pb_column_2_tb_body .et_pb_text_3_tb_body .et_pb_text_inner')
            fecha = fecha_element.text if fecha_element else None
        except:
            fecha = None

        # Extraer el autor con el nuevo selector
        try:
            autor_element = driver.find_element(By.CSS_SELECTOR, 'a.molongui-disabled-link')
            autor = autor_element.text if autor_element else None
        except:
            autor = None
        
        # Extraer el producto con el nuevo selector
        try:
            producto_element = driver.find_element(By.CSS_SELECTOR, '.et_pb_text_0_tb_body .et_pb_text_inner a')
            producto = producto_element.text if producto_element else None
        except:
            producto = None

        return {
            'Título': titulo,
            'Fecha': fecha,
            'Autor': autor,
            'Producto': producto
        }
    except Exception as e:
        print(f"Error al acceder a {url}: {e}")
        return {
            'Título': None,
            'Fecha': None,
            'Autor': None,
            'Producto': None
        }

def actualizar_metadata_selenium(df):
    for index, row in df.iterrows():
        if pd.isna(row['Título']) or row['Título'] == '':
            enlace = row['Enlace']
            metadata = extraer_metadata_selenium(enlace)
            df.at[index, 'Título'] = metadata['Título']
            df.at[index, 'Fecha'] = metadata['Fecha']
            df.at[index, 'Autor'] = metadata['Autor']
            df.at[index, 'Producto'] = metadata['Producto']
    return df

# Ejecutar la actualización en el DataFrame nxxi3
nxxi33 = nxxi3.copy()
nxxi3_actualizado = actualizar_metadata_selenium(nxxi33)

driver.quit()

nxxi3_actualizado

,Título,Enlace,Fecha,Producto,Autor,Contenido,Imagen,Fecha de publicación,Corpus
0,"A CUATRO AÑOS DE LA REVUELTA SOCIAL, VOLVER A ...",https://www.nodoxxi.cl/noticias/2023/a-cuatro-...,18 octubre 2023,Noticias,Camila Miranda,NaN,NaN,2023-10-18,A cuatro años...
1,"EL PAÍS NO NECESITA AL CAE, EL CAE NECESITA SO...",https://www.nodoxxi.cl/noticias/opinion/2023/e...,7 octubre 2023,Noticias,Nodo xxi,NaN,NaN,2023-10-07,En entrevista...
2,CONMEMORACIÓN 50 AÑOS DEL GOLPE DE ESTADO: ¿HA...,https://www.nodoxxi.cl/noticias/2023/conmemora...,23 julio 2023,Noticias,Camila Miranda y Pierina Ferretti,NaN,NaN,2023-07-23,Hace diez año...
3,PRESENTACIÓN DE LA IPN Nº 10.887,https://www.nodoxxi.cl/noticias/2023/presentac...,23 julio 2023,Noticias,Nodo xxi,NaN,NaN,2023-07-23,El miércoles ...
4,DECLARACIÓN INICIATIVAS POPULARES CIUDAD...,https://www.nodoxxi.cl/noticias/2023/declaraci...,11 julio 2023,Noticias,Camila Miranda,NaN,NaN,2023-07-11,Estamos acá r...
...,...,...,...,...,...,...,...,...,...
523,DRA. DANAE SINCLAIRE DE NODO XXI EXPONE EN JOR...,https://www.nodoxxi.cl/otros/2024/dra-danae-si...,15 agosto 2024,Otros,Pierina Ferretti,NaN,NaN,2024-08-15,El pasado jue...
524,NODO XXI REALIZA UN ESTUDIO SOBRE LA EFICIENCI...,https://www.nodoxxi.cl/portada/2024/nodo-xxi-r...,16 diciembre 2024,Otros,Pierina Ferretti,NaN,NaN,2024-12-16,En el marco d...
525,LOS SEIS NOMBRES QUE SE REUNIRÁN A DEBATIR SOB...,https://www.nodoxxi.cl/noticias/2023/los-seis-...,18 enero 2023,Noticias,Pierina Ferretti,NaN,NaN,2023-01-18,La Fundación ...
526,NODO XXI REALIZA ESTUDIO QUE INDAGA EN LA RELA...,https://www.nodoxxi.cl/portada/2024/nodo-xxi-r...,15 diciembre 2024,Otros,Pierina Ferretti,NaN,NaN,2024-12-15,Ganar sin per...


In [75]:
import pandas as pd

# Diccionario de mapeo de meses en español a números
meses_espanol_a_numero = {
    'enero': 1, 'febrero': 2, 'marzo': 3, 'abril': 4, 'mayo': 5, 'junio': 6,
    'julio': 7, 'agosto': 8, 'septiembre': 9, 'octubre': 10, 'noviembre': 11, 'diciembre': 12
}

# Función para convertir la columna 'Fecha' en 'Fecha de publicación'
def convertir_fecha(fecha):
    if not isinstance(fecha, str) or fecha.strip() == '':
        return pd.NaT  # Maneja valores nulos o vacíos
    
    partes = fecha.split()
    if len(partes) != 3:
        return pd.NaT  # Si el formato no es "día mes año", retorna NaT
    
    dia, mes, anio = partes
    mes = meses_espanol_a_numero.get(mes)  # Convertir mes a número
    
    if not mes:
        return pd.NaT  # Si el mes no está en el diccionario, retornar NaT
    
    return pd.to_datetime(f"{anio}-{mes}-{dia}", format='%Y-%m-%d')

In [76]:
nxxi3_actualizado2 = nxxi3_actualizado.copy()
nxxi3_actualizado2['Fecha de publicación'] = nxxi3_actualizado2['Fecha'].apply(convertir_fecha)
nxxi3_actualizado2

,Título,Enlace,Fecha,Producto,Autor,Contenido,Imagen,Fecha de publicación,Corpus
0,"A CUATRO AÑOS DE LA REVUELTA SOCIAL, VOLVER A ...",https://www.nodoxxi.cl/noticias/2023/a-cuatro-...,18 octubre 2023,Noticias,Camila Miranda,NaN,NaN,2023-10-18,A cuatro años...
1,"EL PAÍS NO NECESITA AL CAE, EL CAE NECESITA SO...",https://www.nodoxxi.cl/noticias/opinion/2023/e...,7 octubre 2023,Noticias,Nodo xxi,NaN,NaN,2023-10-07,En entrevista...
2,CONMEMORACIÓN 50 AÑOS DEL GOLPE DE ESTADO: ¿HA...,https://www.nodoxxi.cl/noticias/2023/conmemora...,23 julio 2023,Noticias,Camila Miranda y Pierina Ferretti,NaN,NaN,2023-07-23,Hace diez año...
3,PRESENTACIÓN DE LA IPN Nº 10.887,https://www.nodoxxi.cl/noticias/2023/presentac...,23 julio 2023,Noticias,Nodo xxi,NaN,NaN,2023-07-23,El miércoles ...
4,DECLARACIÓN INICIATIVAS POPULARES CIUDAD...,https://www.nodoxxi.cl/noticias/2023/declaraci...,11 julio 2023,Noticias,Camila Miranda,NaN,NaN,2023-07-11,Estamos acá r...
...,...,...,...,...,...,...,...,...,...
523,DRA. DANAE SINCLAIRE DE NODO XXI EXPONE EN JOR...,https://www.nodoxxi.cl/otros/2024/dra-danae-si...,15 agosto 2024,Otros,Pierina Ferretti,NaN,NaN,2024-08-15,El pasado jue...
524,NODO XXI REALIZA UN ESTUDIO SOBRE LA EFICIENCI...,https://www.nodoxxi.cl/portada/2024/nodo-xxi-r...,16 diciembre 2024,Otros,Pierina Ferretti,NaN,NaN,2024-12-16,En el marco d...
525,LOS SEIS NOMBRES QUE SE REUNIRÁN A DEBATIR SOB...,https://www.nodoxxi.cl/noticias/2023/los-seis-...,18 enero 2023,Noticias,Pierina Ferretti,NaN,NaN,2023-01-18,La Fundación ...
526,NODO XXI REALIZA ESTUDIO QUE INDAGA EN LA RELA...,https://www.nodoxxi.cl/portada/2024/nodo-xxi-r...,15 diciembre 2024,Otros,Pierina Ferretti,NaN,NaN,2024-12-15,Ganar sin per...


In [79]:
import pandas as pd
import re

def limpiar_caracteres(valor):
    if isinstance(valor, str):
        # Elimina caracteres de control excepto tabulación, nueva línea y retorno de carro
        valor = re.sub(r'[\x00-\x09\x0b\x0c\x0e-\x1f\x7f]', '', valor)
        # Reemplaza caracteres específicos que causan problemas comunes
        valor = valor.replace('\xa0', ' ')  # Espacio en blanco invisible
        # Puedes añadir más reemplazos aquí si encuentras otros caracteres problemáticos
    return valor

nxxi3_actualizado_final = nxxi3_actualizado2.applymap(limpiar_caracteres)

In [80]:
nxxi3_actualizado_final.to_excel("NodoXXI.xlsx", index=False)